In [16]:
#!pip install optuna
!pip install flask



# Hyperparameter Tuning

## Using Optuna

In [12]:
import optuna
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
data = load_iris()
X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, test_size=0.2)

def objective(trial):   

    n_estimators = trial.suggest_int('n_estimators', 2, 100)
    max_depth = trial.suggest_int('max_depth', 1, 32)
    clf = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth)

    clf.fit(X_train, y_train)
    preds = clf.predict(X_test)
    accuracy = accuracy_score(y_test, preds)
    return accuracy


In [9]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

print('Best trial:')
trial = study.best_trial
print('  Value: {}'.format(trial.value))
print('  Params: ')
for key, value in trial.params.items():
    print('    {}: {}'.format(key, value))


[I 2025-01-07 20:54:13,684] A new study created in memory with name: no-name-2c7eae80-446e-4329-b733-a33d1b3239f4
[I 2025-01-07 20:54:13,747] Trial 0 finished with value: 1.0 and parameters: {'n_estimators': 68, 'max_depth': 13}. Best is trial 0 with value: 1.0.
[I 2025-01-07 20:54:13,813] Trial 1 finished with value: 0.9333333333333333 and parameters: {'n_estimators': 74, 'max_depth': 3}. Best is trial 0 with value: 1.0.
[I 2025-01-07 20:54:13,900] Trial 2 finished with value: 0.9666666666666667 and parameters: {'n_estimators': 98, 'max_depth': 15}. Best is trial 0 with value: 1.0.
[I 2025-01-07 20:54:13,928] Trial 3 finished with value: 0.9666666666666667 and parameters: {'n_estimators': 22, 'max_depth': 12}. Best is trial 0 with value: 1.0.
[I 2025-01-07 20:54:13,935] Trial 4 finished with value: 1.0 and parameters: {'n_estimators': 2, 'max_depth': 28}. Best is trial 0 with value: 1.0.
[I 2025-01-07 20:54:13,989] Trial 5 finished with value: 0.9 and parameters: {'n_estimators': 63, 

Best trial:
  Value: 1.0
  Params: 
    n_estimators: 68
    max_depth: 13


## Using Scikit-learn’s GridSearchCV

In [10]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [10, 50, 100],
    'max_depth': [None, 10, 20, 30]
}


In [13]:
clf = RandomForestClassifier()
grid_search = GridSearchCV(clf, param_grid, cv=3)
grid_search.fit(X_train, y_train)

print("Best parameters found: ", grid_search.best_params_)


Best parameters found:  {'max_depth': 20, 'n_estimators': 10}


In [18]:
import joblib

# Save the model
joblib.dump(clf, 'model.pkl')


['model.pkl']

# Model Packaging

In [20]:
from flask import Flask, request, jsonify
import joblib

app = Flask(__name__)
model = joblib.load('model.pkl')  # Ensure this path is correct

@app.route('/predict', methods=['POST'])
def predict():
    data = request.get_json()
    prediction = model.predict([data['input']])
    return jsonify({'prediction': prediction.tolist()})

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.1.4:5000
Press CTRL+C to quit
127.0.0.1 - - [07/Jan/2025 21:04:54] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [07/Jan/2025 21:04:58] "GET / HTTP/1.1" 404 -
